In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib

In [3]:
# 加載模型
regressor = joblib.load('svr_model.joblib')  # 從文件中加載模型
scaler = joblib.load('scaler.joblib')        # 加載標準化對象

# === Step 1: 讀取資料 ===
#D:/SCU/eyemyself/資料前處理/Mandy_encoding.csv
data = pd.read_csv('/content/活頁簿1.csv')


In [4]:
# === Step 2: 新增轉變標籤 ===
data['Transition'] = data.groupby('USER_ID_HASH')['Exhausted_state'].transform(
    lambda x: (x.shift() == 0) & (x == 1)
).astype(int)


# === Step 3: 計算首次轉變時間差 ===
def calculate_time_diff(df):
    start_time = df['time'].iloc[0]
    if df['Transition'].sum() > 0:  # 如果有轉變
        transition_time = df.loc[df['Transition'] == 1, 'time'].iloc[0]
        df['time_diff_to_first_exhausted'] = transition_time - start_time
    else:
        df['time_diff_to_first_exhausted'] = None
    return df

data = data.groupby('USER_ID_HASH', group_keys=False, as_index=False).apply(calculate_time_diff)

# === Step 4: 計算移動特徵 ===
def add_moving_features(df):
    df['distance_moving_avg'] = df.groupby('USER_ID_HASH')['distance'].transform(
        lambda x: x.rolling(window=5, min_periods=1).mean()
    )
    df['brightness_moving_avg'] = df.groupby('USER_ID_HASH')['brightness_x'].transform(
        lambda x: x.rolling(window=5, min_periods=1).mean()
    )
    df['distance_variance'] = df.groupby('USER_ID_HASH')['distance'].transform(
        lambda x: x.rolling(window=5, min_periods=1).var()
    )
    return df

data = add_moving_features(data)

# === Step 5: 過濾資料 ===
def filter_first_transition(df):
    if df['Transition'].sum() == 0:  # 若無轉變，保留所有資料
        return df
    first_transition_index = df[df['Transition'] == 1].index[0]
    return df[df.index <= first_transition_index]

filtered_data = data.groupby('USER_ID_HASH', group_keys=False).apply(filter_first_transition).reset_index(drop=True)


<ipython-input-4-60d7643b3ece>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('USER_ID_HASH', group_keys=False, as_index=False).apply(calculate_time_diff)
<ipython-input-4-60d7643b3ece>:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_data = data.groupby('USER_ID_HASH', group_keys=False).apply(filter_first_transition).reset_index(drop=True)


In [5]:
# 計算相關矩陣
correlation_matrix = filtered_data.corr()
correlation_with_target = correlation_matrix['time_diff_to_first_exhausted'].drop('time_diff_to_first_exhausted')
print("與 time_diff_to_first_exhausted 的相關係數：")
print(correlation_with_target)

# 1. 選擇要用於填補的特徵
features_to_impute = ['time_diff_to_first_exhausted', 'brightness_x' , 'distance_ratio', 'brightness_y',
                      'blink_y','question_7','question_9','question_10','question_2_其他',
                      'question_2_完成學校作業','brightness_moving_avg','distance_variance',
                      ]
imputer = KNNImputer(n_neighbors=10)#建立 KNNImputer 模型
filtered_data[features_to_impute] = imputer.fit_transform(filtered_data[features_to_impute]) #使用 KNNImputer 填補缺失值


ValueError: could not convert string to float: 'Joy'

In [6]:
# === Step 6: 特徵選擇與清理 === 需要更改
feature_cols = ['USER_ID_HASH',
 'age','gender',
 'right_eye_condition','right_eye_degree','right_eye_shine','right_eye_shine_degree',
 'left_eye_condition','left_eye_degree','left_eye_shine','left_eye_shine_degree',
 'eye_situation_value1','eye_situation_value2','eye_situation_value3','eye_situation_value4','eye_situation_value5',
 'use_situation1','use_situation3','habit1',
 'distance','brightness_x','blink_x','state','Exhausted_state','time',
 'distance_ratio','brightness_y','blink_y','blink_num',
 'question_5','question_6','question_7','question_8','question_9','question_10','question_11',
 'use_situation2_3小時以內','use_situation2_3至6小時','use_situation2_6-9小時','use_situation2_9-12小時','use_situation2_12小時以上',
 'use_situation_value4_電腦自動調整','use_situation_value4_不常調整','use_situation_value4_每次使用都會調整',
 'use_situation_value5_僅室內共用燈光','use_situation_value5_僅室內專用燈光','use_situation_value5_室內共用與專用燈光皆有','use_situation_value5_戶外','use_situation_value5_光線明顯不足之環境','use_situation_value5_其他',
 'habit2_無','habit2_半年一次','habit2_一年一次','habit2_更頻繁',
 'habit3_低於4小時','habit3_4至6小時','habit3_6至8小時','habit3_高於8小時',
 'habit4_0或1次','habit4_2或3次','habit4_4或5次','habit4_6次以上',
 'habit5_無休息','habit5_1小時內','habit5_1-2小時','habit5_2-3小時','habit5_3-4小時','habit5_4-5小時','habit5_5小時以上',
 'habit6_10分鐘內','habit6_11-30分鐘','habit6_31-60分鐘','habit6_60分鐘以上',
 'habit7_眼部運動','habit7_閉目養神','habit7_其他','habit7_閉目養神, 眼部運動','habit7_閉目養神, 其他','habit7_眼部運動, 其他','habit7_閉目養神, 眼部運動, 其他',
 'question_1_電腦','question_1_手機','question_1_平板','question_1_其他',
 'question_2_工作/實習用途','question_2_聆聽線上課程', 'question_2_完成學校作業','question_2_打電腦遊戲','question_2_觀看影音串流平台(如youtube)','question_2_回復訊息文字','question_2_其他',
 'question_3_僅室內共用燈光','question_3_僅室內專用燈光','question_3_戶外','question_3_光線明顯不足之環境','question_3_其他',
 'question_4_無','question_4_配戴眼鏡','question_4_配戴隱形眼鏡',
 'Transition','time_diff_to_first_exhausted','distance_moving_avg','brightness_moving_avg','distance_variance']

missing_values = filtered_data.isnull().sum()

#filtered_data['habit2']=0


In [7]:
filtered_data = filtered_data.dropna(subset=feature_cols + ['Transition'])  # 移除缺失值
print(filtered_data.isnull().sum())  # 檢查每列缺失值
print((filtered_data == float('inf')).sum())  # 檢查正無窮
print((filtered_data == float('-inf')).sum())

KeyError: ['age', 'use_situation2_3小時以內', 'use_situation2_3至6小時', 'use_situation2_6-9小時', 'use_situation2_9-12小時', 'use_situation2_12小時以上', 'use_situation_value4_電腦自動調整', 'use_situation_value4_不常調整', 'use_situation_value4_每次使用都會調整', 'use_situation_value5_僅室內共用燈光', 'use_situation_value5_僅室內專用燈光', 'use_situation_value5_室內共用與專用燈光皆有', 'use_situation_value5_戶外', 'use_situation_value5_光線明顯不足之環境', 'use_situation_value5_其他', 'habit2_無', 'habit2_半年一次', 'habit2_一年一次', 'habit2_更頻繁', 'habit3_低於4小時', 'habit3_4至6小時', 'habit3_6至8小時', 'habit3_高於8小時', 'habit4_0或1次', 'habit4_2或3次', 'habit4_4或5次', 'habit4_6次以上', 'habit5_無休息', 'habit5_1小時內', 'habit5_1-2小時', 'habit5_2-3小時', 'habit5_3-4小時', 'habit5_4-5小時', 'habit5_5小時以上', 'habit6_10分鐘內', 'habit6_11-30分鐘', 'habit6_31-60分鐘', 'habit6_60分鐘以上', 'habit7_眼部運動', 'habit7_閉目養神', 'habit7_其他', 'habit7_閉目養神, 眼部運動', 'habit7_閉目養神, 其他', 'habit7_眼部運動, 其他', 'habit7_閉目養神, 眼部運動, 其他', 'question_1_電腦', 'question_1_手機', 'question_1_平板', 'question_1_其他', 'question_2_工作/實習用途', 'que

In [8]:
X_new = filtered_data[feature_cols]
y_new = filtered_data['time_diff_to_first_exhausted']

KeyError: "['age', 'use_situation2_3小時以內', 'use_situation2_3至6小時', 'use_situation2_6-9小時', 'use_situation2_9-12小時', 'use_situation2_12小時以上', 'use_situation_value4_電腦自動調整', 'use_situation_value4_不常調整', 'use_situation_value4_每次使用都會調整', 'use_situation_value5_僅室內共用燈光', 'use_situation_value5_僅室內專用燈光', 'use_situation_value5_室內共用與專用燈光皆有', 'use_situation_value5_戶外', 'use_situation_value5_光線明顯不足之環境', 'use_situation_value5_其他', 'habit2_無', 'habit2_半年一次', 'habit2_一年一次', 'habit2_更頻繁', 'habit3_低於4小時', 'habit3_4至6小時', 'habit3_6至8小時', 'habit3_高於8小時', 'habit4_0或1次', 'habit4_2或3次', 'habit4_4或5次', 'habit4_6次以上', 'habit5_無休息', 'habit5_1小時內', 'habit5_1-2小時', 'habit5_2-3小時', 'habit5_3-4小時', 'habit5_4-5小時', 'habit5_5小時以上', 'habit6_10分鐘內', 'habit6_11-30分鐘', 'habit6_31-60分鐘', 'habit6_60分鐘以上', 'habit7_眼部運動', 'habit7_閉目養神', 'habit7_其他', 'habit7_閉目養神, 眼部運動', 'habit7_閉目養神, 其他', 'habit7_眼部運動, 其他', 'habit7_閉目養神, 眼部運動, 其他', 'question_1_電腦', 'question_1_手機', 'question_1_平板', 'question_1_其他', 'question_2_工作/實習用途', 'qu

In [ ]:
# === Step 3: 標準化特徵 ===
X_new_scaled = scaler.transform(X_new)  # 使用保存的標準化對象進行轉換

# === Step 4: 使用模型進行預測 ===
y_pred_new = regressor.predict(X_new_scaled)

# === Step 5: 評估模型 ===
mae = mean_absolute_error(y_new, y_pred_new)
mse = mean_squared_error(y_new, y_pred_new)
rmse = np.sqrt(mse)
r2 = r2_score(y_new, y_pred_new)

# 輸出評估結果
print("新數據的評估結果：")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- habit2
Feature names seen at fit time, yet now missing:
- USER_ID_HASH
- age
- habit2_一年一次
- habit2_半年一次
- habit2_更頻繁
- ...
